In [ ]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
import graphviz

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Temurin-17.0.12+7 (build 17.0.12+7, mixed mode, sharing)
  Starting server from C:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Huawei\AppData\Local\Temp\tmpqqklt21t
  JVM stdout: C:\Users\Huawei\AppData\Local\Temp\tmpqqklt21t\h2o_Huawei_started_from_python.out
  JVM stderr: C:\Users\Huawei\AppData\Local\Temp\tmpqqklt21t\h2o_Huawei_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Asia/Kuala_Lumpur
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 24 days
H2O_cluster_name:,H2O_from_python_Huawei_25bvjv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.961 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Initialize H2O
h2o.init()

In [ ]:
# Load datasets
X_train = pd.read_csv("../04_modelling/dataset/X_train.csv")
y_train = pd.read_csv("../04_modelling/dataset/y_train.csv")
X_val = pd.read_csv("../04_modelling/dataset/X_val.csv")
y_val = pd.read_csv("../04_modelling/dataset/y_val.csv")
X_test = pd.read_csv("../04_modelling/dataset/X_test.csv")
y_test = pd.read_csv("../04_modelling/dataset/y_test.csv")

In [ ]:
# Combine training features and target for H2O AutoML
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)


In [ ]:
# Convert to H2O Frames
train_h2o = h2o.H2OFrame(train_df)
val_h2o = h2o.H2OFrame(val_df)
test_h2o = h2o.H2OFrame(test_df)


In [ ]:
# Define target and features
target = "yearly_compensation"
features = train_h2o.columns
if target in features:
    features.remove(target)

In [3]:
# Train AutoML model
aml = H2OAutoML(max_models=10, seed=42, include_algos=["GBM", "DRF"])
aml.train(x=features, y=target, training_frame=train_h2o)


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_1_20241226_221209


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    273                273                         126891                 7            15           13.1795       20            36            32.0623

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.4540596733543416
RMSE: 0.6738395011828422
MAE: 0.5159481556439183
RMSLE: NaN
Mean Residual Deviance: 0.4540596733543416

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 3.521877414304122
RMSE: 1.8766665698264362
MAE: 1.3748741818201329
RMSLE: NaN
Mean Residual Deviance: 3.521877414304122

Cross-Validation Metrics Summary: 
                        mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
aic                     nan       0           nan           nan           nan           nan           nan
loglikelihood           nan       0           nan           nan           nan           nan           nan
mae                     1.37486   0.0882278   1.3867        1.3488        1.34459       1.51629       1.27791
mean_residual_deviance  3.52179   0.419183    3.56328       3.46955       3.36294       4.18051       3.03265
mse                     3.52179   0.419183    3.56328       3.46955       3.36294       4.18051       3.03265
r2                      0.938618  0.00525845  0.936474      0.93936       0.940009      0.931428      0.945817
residual_deviance       3.52179   0.419183    3.56328       3.46955       3.36294       4.18051       3.03265
rmse                    1.87405   0.110241    1.88767       1.86267       1.83383       2.04463       1.74145
rmsle                   nan       0           nan           nan           nan           nan           nan

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------
     2024-12-26 22:12:39  29.865 sec  0.0                7.575565475958788   6.835998278025935   57.3891922805387
     2024-12-26 22:12:39  30.071 sec  5.0                5.881045020825763   5.231735931689277   34.586690536979496
     2024-12-26 22:12:39  30.184 sec  10.0               4.7589262006749475  4.125768194095537   22.64737858347049
     2024-12-26 22:12:39  30.294 sec  15.0               4.062395495119005   3.435262020963259   16.503057158763184
     2024-12-26 22:12:39  30.430 sec  20.0               3.5096070948372913  2.888348376831198   12.317341960132252
     2024-12-26 22:12:40  30.525 sec  25.0               3.1191229920211554  2.496218423550001   9.728928239355005
     2024-12-26 22:12:40  30.637 sec  30.0               2.815222604949222   2.204373004807889   7.925478315417084
     2024-12-26 22:12:40  30.777 sec  35.0               2.5719044152132144  1.9738715608401405  6.614692320993226
     2024-12-26 22:12:40  30.848 sec  40.0               2.3641333422608284  1.7950225574209915  5.589126459989355
     2024-12-26 22:12:40  30.987 sec  45.0               2.1935376864538583  1.6504052431537766  4.8116075818933455
---  ---                  ---         ---                ---                 ---                 ---
     2024-12-26 22:12:44  34.684 sec  230.0              0.7735444618069036  0.5920721162777626  0.5983710343921322
     2024-12-26 22:12:44  34.818 sec  235.0              0.7608790035627157  0.5826484054931117  0.578936858062591
     2024-12-26 22:12:44  34.946 

In [4]:
# Display leaderboard
print("H2O AutoML Leaderboard:")
print(aml.leaderboard)


H2O AutoML Leaderboard:
model_id                                        rmse       mse      mae       rmsle    mean_residual_deviance
GBM_1_AutoML_1_20241226_221209               1.87667   3.52188  1.37487  nan                          3.52188
GBM_grid_1_AutoML_1_20241226_221209_model_1  2.14408   4.59709  1.56718  nan                          4.59709
GBM_2_AutoML_1_20241226_221209               2.19528   4.81924  1.59312  nan                          4.81924
GBM_5_AutoML_1_20241226_221209               2.219     4.92397  1.60779  nan                          4.92397
GBM_grid_1_AutoML_1_20241226_221209_model_2  2.22596   4.95491  1.57824  nan                          4.95491
GBM_3_AutoML_1_20241226_221209               2.25622   5.09054  1.64942  nan                          5.09054
GBM_4_AutoML_1_20241226_221209               2.34223   5.48606  1.71535  nan                          5.48606
DRF_1_AutoML_1_20241226_221209               3.1552    9.95527  2.46362    0.580605             

In [5]:
# Extract and visualize the best model (if Decision Tree is included)
model = aml.get_best_model()
if 'drf' in model.model_id:
    tree = h2o.get_model_tree(model, tree_number=0)
    print("Decision Tree Details:")
    print(tree)


In [6]:
# Train and visualize Decision Tree using sklearn
clf = DecisionTreeClassifier(max_depth=3, criterion='gini', random_state=42)
clf.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=3, random_state=42)

In [ ]:
# Export the decision tree
# dot_data = export_graphviz(clf, out_file=None, feature_names=X_train.columns, filled=True, rounded=True, special_characters=True)
# graph = graphviz.Source(dot_data)
# graph.render("DecisionTree")

In [8]:
# Model evaluation
print("Evaluation on Test Set:")
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Evaluation on Test Set:
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.62      0.57       122
           1       0.00      0.00      0.00        49
           2       0.96      0.54      0.69        79
           3       0.28      0.91      0.42        74
           4       0.00      0.00      0.00        49
           5       0.35      0.53      0.42        51
           6       0.00      0.00      0.00        45
           7       0.21      0.42      0.28        36
           8       0.00      0.00      0.00        52
           9       0.00      0.00      0.00        14
          10       0.26      0.45      0.33        47
          11       0.00      0.00      0.00         6
          12       0.00      0.00      0.00        26
          13       0.32      0.79      0.45        73
          14       0.00      0.00      0.00         7
          15       0.00      0.00      0.00        27
          16       0.00      0.00 

c:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Huawei\OneDrive - Universiti Malaya\Desktop\SEMESTER 7\WIE3007_Data-Mining\Group Project\data-mining-warehousing-wages-analysis

In [9]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[76  0  0  1  0  0  0 13  0  0  0  0  0  2  0  0  0  0  0  0  0 30  0  0
   0]
 [26  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0 17  0  0
   0]
 [ 1  0 43  0  0 26  0  0  0  0  3  0  0  6  0  0  0  0  0  0  0  0  0  0
   0]
 [ 4  0  0 67  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0
   0]
 [ 6  0  0 38  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5  0  0
   0]
 [ 2  0  2  0  0 27  0  0  0  0 18  0  0  2  0  0  0  0  0  0  0  0  0  0
   0]
 [ 2  0  0 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 27  0  0
   0]
 [ 1  0  0  0  0  0  0 15  0  0  0  0  0  0  0  0  0  0  0  0  0 20  0  0
   0]
 [ 0  0  0  0  0 15  0  0  0  0 23  0  0 14  0  0  0  0  0  0  0  0  0  0
   0]
 [ 0  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0
   0]
 [ 0  0  0  0  0  6  0  0  0  0 21  0  0 20  0  0  0  0  0  0  0  0  0  0
   0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5  0  0
   0]
 [ 0  0  0  0  0  0  0